In [7]:
import os
import json
import sys
import pandas as pd
import torch

PROJECT_ROOT = "/media/ttoxopeus/basic_UNet"

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"✅ Added project root to sys.path:\n   {PROJECT_ROOT}")
else:
    print(f"ℹ️ Project root already in sys.path:\n   {PROJECT_ROOT}")


from src.training.eval import evaluate
from src.models.unet import UNet
from src.pruning.model_inspect import model_to_dataframe_with_l1, get_pruning_masks_blockwise, compute_actual_prune_ratios, compute_l1_norms, compute_l1_stats, inspect_model_l1
from src.pruning.rebuild import rebuild_pruned_unet, find_prev_conv_name

ℹ️ Project root already in sys.path:
   /media/ttoxopeus/basic_UNet


In [8]:
# Load the model
model = UNet(in_ch=1, out_ch=4, enc_features=[64, 128, 256, 512], dec_features=None, bottleneck_out=None)
state = torch.load("/media/ttoxopeus/basic_UNet/results/UNet_ACDC/exp35/baseline/training/final_model.pth", map_location="cpu")
model.load_state_dict(state)
model.eval()
print("✅ Model loaded successfully.")

RuntimeError: Error(s) in loading state_dict for UNet:
	Unexpected key(s) in state_dict: "encoders.4.net.0.weight", "encoders.4.net.1.weight", "encoders.4.net.1.bias", "encoders.4.net.1.running_mean", "encoders.4.net.1.running_var", "encoders.4.net.1.num_batches_tracked", "encoders.4.net.3.weight", "encoders.4.net.4.weight", "encoders.4.net.4.bias", "encoders.4.net.4.running_mean", "encoders.4.net.4.running_var", "encoders.4.net.4.num_batches_tracked", "decoders.8.weight", "decoders.8.bias", "decoders.9.net.0.weight", "decoders.9.net.1.weight", "decoders.9.net.1.bias", "decoders.9.net.1.running_mean", "decoders.9.net.1.running_var", "decoders.9.net.1.num_batches_tracked", "decoders.9.net.3.weight", "decoders.9.net.4.weight", "decoders.9.net.4.bias", "decoders.9.net.4.running_mean", "decoders.9.net.4.running_var", "decoders.9.net.4.num_batches_tracked". 
	size mismatch for bottleneck.net.0.weight: copying a param with shape torch.Size([2048, 1024, 3, 3]) from checkpoint, the shape in cu

In [9]:
df_summary = inspect_model_l1(model, save_dir="./l1_analysis")
pd.set_option("display.max_rows", None)
display(df_summary)

,Layer,Type,Shape,In Ch,Out Ch,Num Params,Mean L1,Min L1,Max L1,L1 Std,Block Ratio,Post-Prune Ratio
0,encoders.0.net.0,Conv2d,"(64, 1, 3, 3)",1,64,576,1.497146,0.723068,2.381193,0.295007,None,None
1,encoders.0.net.3,Conv2d,"(64, 64, 3, 3)",64,64,36864,13.766562,12.143190,17.050581,1.217917,None,None
2,encoders.1.net.0,Conv2d,"(128, 64, 3, 3)",64,128,73728,14.203245,12.271872,17.570244,1.147970,None,None
3,encoders.1.net.3,Conv2d,"(128, 128, 3, 3)",128,128,147456,21.472519,18.238306,28.903608,2.274730,None,None
4,encoders.2.net.0,Conv2d,"(256, 128, 3, 3)",128,256,294912,21.079504,18.116941,28.666965,1.997199,None,None
5,encoders.2.net.3,Conv2d,"(256, 256, 3, 3)",256,256,589824,33.840195,27.027081,45.776711,4.228486,None,None
6,encoders.3.net.0,Conv2d,"(512, 256, 3, 3)",256,512,1179648,38.625061,29.030735,62.542168,6.329008,None,None
7,encoders.3.net.3,Conv2d,"(512, 512, 3, 3)",512,512,2359296,72.066132,45.454155,122.820198,12.509732,None,None
8,bottleneck.net.0,Conv2d,"(1024, 512, 3, 3)",512,1024,4718592,33.947380,33.087589,34.967125,0.281354,None,None
9,bottleneck.net.3,Conv2d,"(1024, 1024, 3, 3)",1024,1024,9437184,47.987492,47.034153,48.872692,0.300650,None,None


In [10]:
norms = compute_l1_norms(model)
stats = compute_l1_stats(norms)
df = model_to_dataframe_with_l1(model, stats, remove_nan_layers=False)

pd.set_option("display.max_rows", None)
display(df)

,Layer,Type,Shape,In Ch,Out Ch,Num Params,Mean L1,Min L1,Max L1,L1 Std,Block Ratio,Post-Prune Ratio
0,encoders.0.net.0,Conv2d,"(64, 1, 3, 3)",1,64,576,1.497146,0.723068,2.381193,0.295007,None,None
1,encoders.0.net.3,Conv2d,"(64, 64, 3, 3)",64,64,36864,13.766562,12.143190,17.050581,1.217917,None,None
2,encoders.1.net.0,Conv2d,"(128, 64, 3, 3)",64,128,73728,14.203245,12.271872,17.570244,1.147970,None,None
3,encoders.1.net.3,Conv2d,"(128, 128, 3, 3)",128,128,147456,21.472519,18.238306,28.903608,2.274730,None,None
4,encoders.2.net.0,Conv2d,"(256, 128, 3, 3)",128,256,294912,21.079504,18.116941,28.666965,1.997199,None,None
5,encoders.2.net.3,Conv2d,"(256, 256, 3, 3)",256,256,589824,33.840195,27.027081,45.776711,4.228486,None,None
6,encoders.3.net.0,Conv2d,"(512, 256, 3, 3)",256,512,1179648,38.625061,29.030735,62.542168,6.329008,None,None
7,encoders.3.net.3,Conv2d,"(512, 512, 3, 3)",512,512,2359296,72.066132,45.454155,122.820198,12.509732,None,None
8,bottleneck.net.0,Conv2d,"(1024, 512, 3, 3)",512,1024,4718592,33.947380,33.087589,34.967125,0.281354,None,None
9,bottleneck.net.3,Conv2d,"(1024, 1024, 3, 3)",1024,1024,9437184,47.987492,47.034153,48.872692,0.300650,None,None
